In [116]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [139]:
def getPage(make, model, page):
    webpage = "https://www.truecar.com/used-cars-for-sale/listings/{}/{}/location-brooklyn-ny/?page={}&searchRadius=25".format(make, model, page)
    page = BeautifulSoup(requests.get(webpage).content, "html.parser")    
    table = page.find_all('li',class_="margin-top-3 d-flex flex-grow col-md-6 col-xl-4")
    return table

In [144]:
def searchTrueCar(make, model, page):
    car_dict = {'id':[], 'year':[], 'make':[], 'model':[], 'trim':[], 'price':[], 
                'rating':[], 'diff':[], 'mileage':[], 'excolor':[], 'intcolor':[], 
                'accidents':[], 'owners':[], 'use':[]}
    for i in range(1, page):
        print('page: ', i)
        results = getPage(make, model, i)
        for car in results:
            try:
                id_ = re.search("(?<=(listing\/))(.*?)(?=(\/2))", car.find('a', class_="linkable order-2 vehicle-card-overlay", href=True)['href']).group()
                car_dict['id'].append(id_)
            except AttributeError:
                print('Out of Listings.')
                break

            year = car.find('span', class_="vehicle-card-year font-size-1").text
            car_dict['year'].append(year)

            makemodel = car.find('span', class_="vehicle-header-make-model text-truncate").text.split(' ')
            car_dict['make'].append(makemodel[0])
            car_dict['model'].append(makemodel[1])

            trim = car.find_all('div', class_="font-size-1 text-truncate")[0].text
            car_dict['trim'].append(trim)
            
            try:
                price = car.find('div', class_="heading-3 margin-y-1 font-weight-bold").text
                car_dict['price'].append(float(re.sub(r'[^\d.]', '', price)))
            except AttributeError:
                car_dict['price'].append('None')
            
            try:
                rating = car.find('span', class_="graph-icon-title margin-left-1 vehicle-card-price-rating-label text-truncate font-weight-bold").text
                car_dict['rating'].append(rating)
            except AttributeError:
                car_dict['rating'].append('None')
                
            try:    
                diff = car.find_all('div', class_="font-size-1 text-truncate")[1].text
                car_dict['diff'].append(diff)
            except AttributeError:
                car_dict['diff'].append('None')
            
            try:
                mileage = car.find_all('div', class_="font-size-1 text-truncate")[2].text
                car_dict['mileage'].append(float(re.sub(r'[^\d.]', '', mileage)))
            except IndexError:
                mileage = car.find_all('div', class_="font-size-1 text-truncate")[1].text
                print(mileage)
                car_dict['mileage'].append(float(re.sub(r'[^\d.]', '', mileage)))

            color = car.find('div', class_="vehicle-card-location font-size-1 margin-top-1 text-truncate").text.split(', ')
            car_dict['excolor'].append(color[0].split(' ')[0])
            car_dict['intcolor'].append(color[1].split(' ')[0])

            history = car.find_all('div', class_="vehicle-card-location font-size-1 margin-top-1")[1].text.split(', ')
            car_dict['accidents'].append(history[0].split(' ')[0])
            car_dict['owners'].append(history[1].split(' ')[0])
            car_dict['use'].append(history[2].split(' ')[0])
                    
    return car_dict

In [145]:
corolla = searchTrueCar('toyota', 'corolla', 10)

page:  1
page:  2
page:  3
page:  4
page:  5
page:  6
30,768 miles
page:  7
31,419 miles
page:  8
page:  9


In [146]:
civic = searchTrueCar('honda', 'civic', 10)

page:  1
page:  2
page:  3
page:  4
page:  5
page:  6
page:  7
page:  8
page:  9
342,650 miles


In [147]:
rio = searchTrueCar('kia', 'rio', 10)

page:  1
page:  2
85,359 miles
page:  3
page:  4
page:  5
page:  6
page:  7
page:  8
page:  9


In [148]:
accent = searchTrueCar('hyundai', 'accent', 10)

page:  1
page:  2
page:  3
71,000 miles
83,775 miles
page:  4
page:  5
page:  6
page:  7
page:  8
page:  9


In [149]:
focus = searchTrueCar('ford', 'focus', 10)

page:  1
page:  2
page:  3
126,106 miles
page:  4
page:  5
page:  6
page:  7
page:  8
page:  9


In [150]:
mazda3 = searchTrueCar('mazda', 'mazda3', 10)

page:  1
page:  2
108,834 miles
page:  3
page:  4
page:  5
page:  6
49,296 miles
page:  7
page:  8
page:  9


In [151]:
optima = searchTrueCar('kia', 'optima', 10)

page:  1
page:  2
page:  3
page:  4
page:  5
page:  6
page:  7
page:  8
page:  9


In [152]:
list_ = [corolla, civic, rio, accent, focus, mazda3, optima]
dfs = []
for c in list_:
    temp = pd.DataFrame.from_dict(c)
    print(len(temp))
    dfs.append(temp)

df = pd.DataFrame()
df = pd.concat(dfs)
df.head()

254
273
45
72
110
157
179


,id,year,make,model,trim,price,rating,diff,mileage,excolor,intcolor,accidents,owners,use
0,5YFBURHE7JP813733,2018,Toyota,Corolla,SE Manual,19499.0,Great Price,$584 off avg. list price,56465.0,Silver,Black,No,1,Personal
1,JTDEPRAE0LJ063808,2020,Toyota,Corolla,LE CVT,22532.0,Great Price,$81 off avg. list price,28549.0,Black,Gray,No,1,Personal
2,1NXBU4EE6AZ307073,2010,Toyota,Corolla,LE Automatic,16990.0,Great Price,$28 off avg. list price,11843.0,Silver,Gray,No,1,Personal
3,5YFEPRAEXLP119569,2020,Toyota,Corolla,LE CVT,21097.0,Great Price,$527 off avg. list price,51363.0,Silver,Unknown,No,1,Personal
4,5YFEPRAE9LP061583,2020,Toyota,Corolla,LE CVT,16900.0,Excellent Price,"$1,472 off avg. list price",59237.0,Blue,Black,1,1,Personal
